<h3><b>Get Microbiom Data

In [171]:
%pip install gdown --quiet

Note: you may need to restart the kernel to use updated packages.


In [172]:
import re
import gdown
import pandas as pd

In [173]:
file_id = '1kt7l75LsKXrQjykGQLppLRIpDrb2tp4D'
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, 'microbiom_data.csv', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1kt7l75LsKXrQjykGQLppLRIpDrb2tp4D
To: c:\Users\artis\OneDrive\Документы\Работа\ML models\MicrobesAndGlucouseAnalysis\microbiom_data.csv
100%|██████████| 3.04M/3.04M [00:00<00:00, 35.7MB/s]


'microbiom_data.csv'

In [174]:
original_micr_data = pd.read_csv('microbiom_data.csv', dtype='string', encoding='UTF-8')

In [175]:
type_constrained_micr_data = pd.concat([original_micr_data.iloc[0:413,0].astype('string'),
                                        original_micr_data.iloc[0:413,1].astype('int'),
                                        original_micr_data.iloc[0:413,2:].astype('float')], axis=1)

In [176]:
numerical_micr_data = type_constrained_micr_data.copy()
sample = numerical_micr_data.pop('Sample')
visit = numerical_micr_data.pop('Visit')

In [177]:
N = sample.apply(lambda row: int(row[0:3]))
N.name = 'N'

In [178]:
def read_line(file):
    while True:
        line = file.readline()
        if not line:
            break
        yield line

In [179]:
def read_files(files):
    LefSe_otu = []
    for file_name in files:
        try:
            with open(file_name, mode='r', encoding='UTF-8') as file:
                for line in read_line(file):
                    bacteria = re.findall(r'\bOTU_\d+\b', line)
                    if bacteria:
                        LefSe_otu.append(*bacteria)
        except FileNotFoundError:
            print(f"File '{file_name}' not found.")
        except IOError as e:
            print(f"Error reading file '{file_name}': {e}")
    return LefSe_otu

In [180]:
LefSe_files = ['LefSe_above_median_BgMax_20_01.tsv', 'LefSe_above_median_iauc120_20_01.tsv']
selected_otu = read_files(LefSe_files)
otu = numerical_micr_data.loc[:, selected_otu]

In [181]:
micr_data = pd.concat([N, visit, otu], axis=1)
micr_data.set_index(['N', 'Visit'], inplace=True)

In [182]:
micr_data = micr_data.loc[(micr_data.index.get_level_values('Visit') == 99) | 
                          (micr_data.index.get_level_values('Visit') == 146)]
micr_data = micr_data.reset_index(level='Visit', drop=True)

In [183]:
micr_data.sort_index()

,OTU_241,OTU_187,OTU_197,OTU_587,OTU_609,OTU_439,OTU_529,OTU_51,OTU_68,OTU_79,...,OTU_338,OTU_305,OTU_312,OTU_496,OTU_420,OTU_193,OTU_454,OTU_399,OTU_257,OTU_337
N,,,,,,,,,,,,,,,,,,,,,
281,0.0,23.0,35.0,0.0,10.0,6.0,0.0,0.0,19.0,0.0,...,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0
288,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,15.0,...,3.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,14.0,0.0
289,0.0,0.0,58.0,0.0,61.0,0.0,0.0,0.0,66.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,0.0
290,0.0,0.0,0.0,111.0,0.0,0.0,0.0,151.0,107.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,35.0,0.0
291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,92.0,0.0,...,8.0,30.0,0.0,0.0,3.0,0.0,0.0,0.0,36.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,0.0,106.0,0.0,0.0,0.0,0.0,0.0,988.0,0.0,205.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,156.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,67.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3><b>Get Clinical Data

In [184]:
with open('clinical_param.txt', mode='r', encoding='UTF-8') as file:
    clinical_param = file.read().split(', ')

In [185]:
original_clinical_data = pd.read_csv('clinical_datа.csv', index_col='N', usecols=['N', *clinical_param], dtype='float', encoding='UTF-8')

In [186]:
clinical_data = original_clinical_data.loc[original_clinical_data['quality_cgm1']==0, clinical_param]
clinical_data.drop(columns=['quality_cgm1'], axis=1, inplace=True)

In [187]:
clinical_data.drop(labels=[712, 724], axis=0, inplace=True)

In [188]:
nans_percentages = clinical_data.isna().mean()
more_than_20 = nans_percentages[nans_percentages >=.2].index
clinical_data.drop(labels=more_than_20, axis=1, inplace=True)

In [189]:
clinical_data.sort_index()

,срок_берем1,диета_до_1визита,Глюкоза_нт_общая,прибавка_m1,отеки1,АД_сист1,АД_диаст1,N_беременностей,N_родов,N_невынашивание,...,кетоны_V1,HbA1C_V1,инсулин_V1,лептин_V1,ФР_V1,БОМК_визит1,Хол_V1,ТГ_V1,ЛПВП_V1,АБ_бер_ть
N,,,,,,,,,,,,,,,,,,,,,
78.0,29.0,1.0,5.26,6.0,0.0,120.0,80.0,2.0,1.0,0.0,...,0.0,5.58,60.20,8.34,209.46,NaN,7.93,1.37,2.71,1.0
166.0,33.0,0.0,4.60,8.0,0.0,110.0,70.0,1.0,0.0,0.0,...,0.0,4.90,96.80,6.71,208.52,NaN,9.80,3.81,2.44,0.0
198.0,24.0,0.0,5.37,3.0,1.0,120.0,80.0,3.0,1.0,0.0,...,0.0,5.40,152.20,48.98,192.56,NaN,6.03,2.21,2.12,0.0
203.0,26.0,1.0,4.90,2.2,0.0,122.0,83.0,2.0,1.0,0.0,...,1.0,4.90,34.20,10.86,216.44,NaN,8.75,2.03,2.54,0.0
212.0,28.0,0.0,4.18,9.0,0.0,110.0,70.0,1.0,0.0,0.0,...,0.0,4.80,58.30,23.98,223.01,NaN,8.01,2.25,2.08,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900.0,16.0,1.0,5.40,0.4,0.0,127.0,90.0,3.0,1.0,1.0,...,0.0,5.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
901.0,27.0,1.0,5.20,10.0,0.0,100.0,55.0,2.0,0.0,0.0,...,0.0,4.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
903.0,26.0,1.0,6.90,0.0,0.0,122.0,77.0,3.0,1.0,1.0,...,0.0,5.10,69.03,56.35,166.00,0.08,4.83,2.35,1.41,0.0


<h3><b>Get monitoring Data

In [190]:
original_monitoring_data = pd.read_csv('monitoring_data.csv', index_col='N', dtype='float', encoding='UTF-8')

In [191]:
nans_percentages = original_monitoring_data.isna().mean()
more_than_20 = nans_percentages[nans_percentages >= .2].index
monitoring_data = original_monitoring_data.drop(labels=more_than_20, axis=1)

In [192]:
monitoring_data.sort_index()

,preg_week,meal_type_n,gi,gl,carbo,prot,fat,kcal,water,mds,...,BGb15,BGb10,BGb5,BG0,BG60,BGMax,AUC60,AUC120,iAUC60,iAUC120
N,,,,,,,,,,,,,,,,,,,,,
77.0,26.0,2.0,61.3,27.6,45.00,19.90,42.20,642.30,342.27,8.00,...,4.3,4.6,4.9,5.4,7.1,7.5,7.0,6.9,1.6,1.5
77.0,26.0,3.0,36.3,10.7,29.50,47.90,12.50,432.10,389.52,22.24,...,4.9,4.9,4.8,4.9,4.9,5.3,5.1,5.1,0.2,0.2
77.0,27.0,1.0,34.8,10.2,29.40,40.60,14.00,414.70,366.38,20.58,...,5.2,5.2,5.2,5.1,6.0,6.6,5.6,5.9,0.5,0.8
77.0,27.0,2.0,50.3,42.6,75.30,24.20,31.90,691.00,349.60,8.45,...,5.4,5.4,5.3,5.3,8.0,8.3,6.2,7.0,0.9,1.7
77.0,27.0,3.0,58.9,28.1,47.60,10.20,19.80,415.00,262.25,23.80,...,5.2,5.2,5.1,5.1,7.0,7.1,5.5,6.1,0.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964.0,NaN,4.0,40.0,0.8,2.07,1.19,11.37,107.70,0.20,0.00,...,8.8,8.2,7.2,6.5,6.5,8.2,7.3,6.4,0.8,-0.1
964.0,NaN,4.0,50.1,5.4,10.80,2.10,3.90,87.00,128.40,0.00,...,5.0,5.7,6.2,6.6,5.3,6.8,5.9,5.2,-0.7,-1.4
964.0,NaN,2.0,61.2,89.9,146.97,4.60,2.79,607.52,246.24,0.00,...,4.6,4.4,4.3,4.5,7.3,8.8,6.8,6.5,2.3,2.0


<h3><b>Conjugate tables

In [193]:
data = clinical_data.merge(micr_data, on='N', how='inner')

In [194]:
data = monitoring_data.merge(data, on='N', how='inner')

In [195]:
target_values = ['BG60', 'BGMax', 'AUC60', 'AUC120', 'iAUC60', 'iAUC120']
crucial_features = ['gi', 'gl', 'carbo', 'prot', 'fat', 'BG0']
data.dropna(subset=[*crucial_features, *target_values], inplace=True)

In [196]:
data.sort_index(inplace=True)
data

,preg_week,meal_type_n,gi,gl,carbo,prot,fat,kcal,water,mds,...,OTU_338,OTU_305,OTU_312,OTU_496,OTU_420,OTU_193,OTU_454,OTU_399,OTU_257,OTU_337
N,,,,,,,,,,,,,,,,,,,,,
281.0,22.0,4.0,38.5,3.7,9.6,0.8,0.3,49.0,186.30,9.50,...,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0
281.0,22.0,4.0,35.9,3.7,10.3,2.9,0.8,71.5,327.08,27.30,...,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0
281.0,22.0,3.0,55.1,15.0,36.4,13.6,43.1,593.7,499.42,18.11,...,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0
281.0,22.0,4.0,20.4,6.2,18.2,19.2,4.0,200.0,444.45,34.82,...,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0
281.0,23.0,4.0,38.4,8.0,20.9,1.8,0.7,102.0,208.65,19.53,...,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873.0,29.0,4.0,30.6,6.4,20.9,9.3,3.3,161.8,317.89,20.93,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
873.0,29.0,2.0,56.8,11.3,20.0,8.7,11.0,216.8,493.06,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
873.0,29.0,4.0,51.6,12.3,19.5,5.9,14.9,228.6,390.38,16.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3><b>Data description

In [197]:
microbial_sequences = original_micr_data.iloc[413, 2:]
otu_micr_pairs = dict(zip(microbial_sequences.index, microbial_sequences.to_numpy()))
selected_otu_micr_pairs = {key: otu_micr_pairs[key] for key in micr_data.columns}

In [198]:
with open('dataset_description.txt', 'w', encoding='UTF-8') as file:

    file.write(f'Total number of unique patients: {len(data.index.unique())}\n')
    file.write(f'Total number of rows: {data.shape[0]}\n\n')
    
    file.write('FEATURES\n\n')
    file.write('\n'.join(data.columns) + '\n\n')

    file.write('OTU DECIPHER\n\n')
    for key, value in selected_otu_micr_pairs.items():
        file.write(f'{key}: {value}\n')

<h3><b>Export

In [199]:
data.to_pickle('data.pkl')

In [200]:
data.to_csv('data.csv', sep=';', decimal=',', encoding='UTF-8')